In [5]:
import os
os.environ['SIRIUS_URL_CONSTS'] = 'http://127.0.0.1'

In [6]:
%gui qt5
%matplotlib qt5

from functools import partial as _partial
import numpy as np
from scipy import optimize

import matplotlib.pyplot as plt
import matplotlib.gridspec as mpl_gs
import matplotlib.cm as cmap
import matplotlib.colors as mpl_colors
from matplotlib import rcParams
rcParams.update(
    {'font.size': 16,'lines.linewidth': 2, 'axes.grid': True})

import pyaccel
import mathphys
from pymodels import si
from apsuite.dynap import DynapEX
from apsuite.optics_analysis import TuneCorr

In [7]:
model = si.create_accelerator()
model.cavity_on = False
model.radiation_on = False
dynap = DynapEX(model)

In [18]:
tunecorr = TuneCorr(model, 'SI')

In [19]:
tunecorr.correct_parameters([49.186188, 14.2300])

1

In [20]:
twi, _ = pyaccel.optics.calc_twiss(dynap._acc)
twi.mux[-1]/2/np.pi, twi.muy[-1]/2/np.pi

(49.18574477838651, 14.230530180928666)

In [8]:
dynap.params.turn_by_turn = True
dynap.params.x_nrpts = 70
dynap.params.de_nrpts = 70
dynap.params.x_min = -0.012
dynap.params.x_max = 0.00
dynap.params.de_min = -0.05
dynap.params.de_max = 0.05
print(dynap)

nrturns      : 512
turn_by_turn : True
x_nrpts      : 70
de_nrpts     : 70
x_min [m]    : -0.012
x_max [m]    : 0
de_min       : -0.05
de_max       : 0.05
y_offset [m] : 0.0001
xl_off [rad] : 1e-05
yl_off [rad] : 1e-05
intnux       : 49.00 (for graphs)
intnuy       : 14.00 (for graphs)



In [10]:
dynap.do_tracking()

In [11]:
dynap.data

{'x_in': array([[-0.012, -0.012, -0.012, -0.012, -0.012],
        [-0.009, -0.009, -0.009, -0.009, -0.009],
        [-0.006, -0.006, -0.006, -0.006, -0.006],
        [-0.003, -0.003, -0.003, -0.003, -0.003],
        [ 0.   ,  0.   ,  0.   ,  0.   ,  0.   ]]),
 'de_in': array([[-0.05 , -0.025,  0.   ,  0.025,  0.05 ],
        [-0.05 , -0.025,  0.   ,  0.025,  0.05 ],
        [-0.05 , -0.025,  0.   ,  0.025,  0.05 ],
        [-0.05 , -0.025,  0.   ,  0.025,  0.05 ],
        [-0.05 , -0.025,  0.   ,  0.025,  0.05 ]]),
 'rout': array([[[-1.20000000e-02,             nan,             nan, ...,
                      nan,             nan,             nan],
         [-1.20000000e-02,             nan,             nan, ...,
                      nan,             nan,             nan],
         [-1.20000000e-02, -4.94333012e-03,  5.86632628e-03, ...,
                      nan,             nan,             nan],
         ...,
         [ 0.00000000e+00,  9.73335776e-05,  1.60684461e-04, ...,
       

In [12]:
dynap.process_data()

In [9]:
# dynap.save_data('/home/fernando/nominal_ex.pickle')
dynap.load_and_apply('/home/fernando/nominal_ex.pickle')
dynap.process_data()

# Diffusion Plot

In [10]:
bounds = [
    dynap.params.intnux, dynap.params.intnux + 0.5,
    dynap.params.intnuy, dynap.params.intnuy + 0.5]
orders = 6
symmetry = 1
resons = dynap.calc_resonances_for_bounds(bounds, orders=orders, symmetry=symmetry)

In [11]:
map2xy = dynap.map_resons2real_plane(resons, maxdist=1e-3, min_diffusion=1e-5)
numb = [x.size for x in map2xy]
filter_resons = []
for i, (res, nu) in enumerate(zip(resons, numb)):
    if nu > 20:
        print(f'{i:2}: {res}  {nu:3}')
        filter_resons.append(res)

to_remove = [0, 1, 2, 6, 7]
for idx in sorted(to_remove, reverse=True):
    del filter_resons[idx]

print()
for i, reson in enumerate(filter_resons):
    print(f'{reson}')

 9: (3, 2, 176)   28
15: (4, 2, 225)   31
16: (0, 5, 71)   31
19: (2, 4, 155)   30
38: (0, 4, 57)   32
41: (0, 6, 85)   28
52: (-4, 1, -182)   25
70: (-2, 1, -84)   39

(2, 4, 155)
(0, 4, 57)
(0, 6, 85)


In [13]:
f, ax, ay = dynap.make_figure_diffusion(contour=True, resons=filter_resons, maxdist=1e-3, min_diffusion=1e-5)
# f, ax, ay = dynap.make_figure_diffusion(contour=True, resons=filter_resons, maxdist=1e-3)
f.show()
# f = dynap.make_figure_diffusion(contour=False)

# X and Y map to tune plot

In [30]:
f = dynap.make_figure_map_real2tune_planes(resons)